<a href="https://colab.research.google.com/github/mr-cri-spy/NLP-Projects/blob/main/Forward_Backward_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install and Import

In [ ]:
!pip install nltk
import nltk
from nltk.tag import hmm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Step 2: Simulated POS-tagged data (like EM would produce)

In [ ]:
train_data = [
    [('Akki', 'NOUN'), ('runs', 'VERB')],
    [('Gopala', 'NOUN'), ('eats', 'VERB')],
    [('Akki', 'NOUN'), ('writes', 'VERB')],
    [('Gopala', 'NOUN'), ('teaches', 'VERB')],
    [('Akki', 'NOUN'), ('learns', 'VERB')],
    [('Gopala', 'NOUN'), ('reads', 'VERB')],
    [('Akki', 'NOUN'), ('eats', 'VERB')],
    [('Gopala', 'NOUN'), ('writes', 'VERB')], ]

Step 3: Train the HMM model

In [ ]:
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_data)

Step 4: Test the tagger

In [ ]:

test_sentence = ['Akki', 'teaches']
output = tagger.tag(test_sentence)
print("Tagged Output:", output)


Tagged Output: [('Akki', 'NOUN'), ('teaches', 'VERB')]


In [ ]:
print("States (Hidden Tags):", tagger._states)
print("\nTransitions:\n", tagger._transitions)  # Remove parentheses here
print("\nEmissions:\n", tagger._outputs)       # Remove parentheses here

States (Hidden Tags): ['NOUN', 'VERB']

Transitions:
 <ConditionalProbDist with 2 conditions>

Emissions:
 <ConditionalProbDist with 2 conditions>


Forward-Backward implementation using NumPy

In [ ]:
import numpy as np


states = ['NOUN', 'VERB']
observations = ['Akki', 'runs']
obs_map = {'Akki': 0, 'runs': 1}
num_states = len(states)
T = len(observations)

A = np.array([[0.6, 0.4],
              [0.3, 0.7]])


B = np.array([[0.9, 0.1],  # NOUN → Akki, runs
              [0.2, 0.8]]) # VERB → Akki, runs


pi = np.array([0.5, 0.5])

# Forward Algorithm (α)

alpha = np.zeros((T, num_states))

# Initialization
alpha[0] = pi * B[:, obs_map[observations[0]]]

# Recursion
for t in range(1, T):
    for j in range(num_states):
        alpha[t, j] = np.sum(alpha[t-1] * A[:, j]) * B[j, obs_map[observations[t]]]

print("Forward Probabilities (α):\n", alpha)


# Backward Algorithm (β)

beta = np.zeros((T, num_states))
beta[T-1] = np.ones(num_states)

# Recursion
for t in range(T-2, -1, -1):
    for i in range(num_states):
        beta[t, i] = np.sum(A[i, :] * B[:, obs_map[observations[t+1]]] * beta[t+1])

print("\nBackward Probabilities (β):\n", beta)


# γ: Posterior probabilities

gamma = (alpha * beta) / np.sum(alpha[T-1])
print("\nGamma (γ = state probabilities at each time step):\n", gamma)


Forward Probabilities (α):
 [[0.45 0.1 ]
 [0.03 0.2 ]]

Backward Probabilities (β):
 [[0.38 0.59]
 [1.   1.  ]]

Gamma (γ = state probabilities at each time step):
 [[0.74347826 0.25652174]
 [0.13043478 0.86956522]]
